In [0]:
%pylab inline

In [1]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd
import spacy

In [2]:
import tqdm

In [3]:
import random

In [4]:
from spacy.training.example import Example

In [5]:
from spacy.util import minibatch, compounding

In [6]:
saves= dataiku.Folder("saves")
saves_info = saves.get_info()
saves_path=saves_info['path']+'/base_config.cfg'

In [7]:
out_path=saves_info['path']+'/config.cfg'

In [8]:
saves_path

'/apps/dataiku/dss_data/managed_folders/DUMMY/AxgEatK8/base_config.cfg'

In [9]:
out_path

'/apps/dataiku/dss_data/managed_folders/DUMMY/AxgEatK8/config.cfg'

In [10]:
saves_info['path']

'/apps/dataiku/dss_data/managed_folders/DUMMY/AxgEatK8'

In [11]:
nlp = spacy.blank("en")

In [12]:
pip show spacy

Name: spacy
Version: 3.6.1
Summary: Industrial-strength Natural Language Processing (NLP) in Python
Home-page: https://spacy.io
Author: Explosion
Author-email: contact@explosion.ai
License: MIT
Location: /apps/dataiku/dss_data/code-envs/python/SentTransform/lib64/python3.6/site-packages
Requires: catalogue, cymem, jinja2, langcodes, murmurhash, numpy, packaging, pathy, preshed, pydantic, requests, setuptools, smart-open, spacy-legacy, spacy-loggers, srsly, thinc, tqdm, typer, typing-extensions, wasabi
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [13]:
from spacy.tokens import DocBin
db = DocBin() 

In [14]:
test='Show me last 3 months sale of product xyz as bar chart.'

In [15]:
test[30:41]

'product xyz'

In [16]:
test[8:21]

'last 3 months'

In [17]:
test[45:54]

'bar chart'

In [18]:
TRAIN_DATA=[("provide me product xyz orders in last 3 months in bar chart format \r",{"entities":[[11,22,"PRODUCT"],
[33,46,"TIME"],
[50,59,"CHART"]]}),
("Show me last 4 months sale of product pqr as bar chart.\r",{"entities":[[8,21,"TIME"],
[30,41,"CHART"],
[45,54,"CHART"]]}),
("give product abc sales for last 8 months in bar chart\r",{"entities":[[5,16,"PRODUCT"],
[27,40,"TIME"],
[44,53,"CHART"]]}),
("get me in bar chart last 10 monthes of sales for product omc\r",{"entities":[[10,19,"CHART"],
[20,35,"TIME"],
[49,60,"PRODUCT"]]}),
("present me last 2 months of revenue in pie chart for product zzk\r",{"entities":[[11,24,"TIME"],
[39,48,"CHART"],
[53,64,"PRODUCT"]]}),
("show me in violin chart the last 22 months of expenditure for product ppz\r",{"entities":[[11,23,"CHART"],
[28,42,"TIME"],
[62,73,"PRODUCT"]]}),
("provide for product lmp in pie chart last 6 months of sales\r",{"entities":[[12,23,"PRODUCT"],
[27,36,"CHART"],
[37,50,"TIME"]]}),
("get me product lmno sales for last 18 months in violin chart\r",{"entities":[[7,19,"PRODUCT"],
[30,44,"TIME"],
[48,60,"CHART"]]}),
("In bar chart show me last 7 months of loss for product ppr.\r",{"entities":[[3,12,"CHART"],
[21,34,"TIME"],
[47,58,"PRODUCT"]]}),
("For last 56 months get me in pie chart sales of product aab",{"entities":[[4,18,"TIME"],
[29,38,"CHART"],
[48,59,"PRODUCT"]]})]

In [19]:
model = None
output_dir=saves_info['path']
n_iter=25

In [23]:
if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

#set up the pipeline
ner = nlp.create_pipe('ner')
if 'ner' not in nlp.pipe_names:
    #ner = nlp.create_pipe('ner')
    nlp.add_pipe('ner', last=True)
else:
    ner = nlp.get_pipe('ner')

Created blank 'en' model


In [24]:
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for batch in minibatch(TRAIN_DATA, size=compounding(4.0, 16.0, 1.001)):
            for text, annotations in batch:
                #nlp.update(
                #    [text],  
                #    [annotations],  
                #    drop=0.5,  
                #    sgd=optimizer,
                #    losses=losses)
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                nlp.update([example], sgd = optimizer, drop=0.4, losses=losses)
            print(losses)

/apps/dataiku/dss_data/code-envs/python/SentTransform/lib64/python3.6/site-packages/spacy/language.py:1060: DeprecationWarning: [W096] The method `nlp.disable_pipes` is now deprecated - use `nlp.select_pipes` instead.
  warnings.warn(Warnings.W096, DeprecationWarning)
/apps/dataiku/dss_data/code-envs/python/SentTransform/lib64/python3.6/site-packages/spacy/language.py:1279: DeprecationWarning: [W089] As of spaCy v3.0, the `nlp.begin_training` method has been renamed to `nlp.initialize`.
  warnings.warn(Warnings.W089, DeprecationWarning)


{'ner': 48.69428062438965}
{'ner': 94.18872714042664}
{'ner': 113.44986003637314}
{'ner': 31.537646740674973}
{'ner': 54.407584719359875}
{'ner': 63.5409092027694}
{'ner': 17.220817839261144}
{'ner': 32.318819880516}
{'ner': 38.864241727294484}
{'ner': 26.28723132026039}
{'ner': 82.13920145772113}
{'ner': 103.95781522106677}
{'ner': 28.72135057102423}
{'ner': 56.19570752850268}
{'ner': 67.30738716537599}
{'ner': 20.437580887111835}
{'ner': 40.348902545665624}
{'ner': 48.051459858910675}
{'ner': 15.562030771845457}
{'ner': 24.59902745463296}
{'ner': 28.398789211649394}
{'ner': 6.7218087246859}
{'ner': 11.130971840118818}
{'ner': 11.771853810591502}
{'ner': 1.4514382282813862}
{'ner': 6.093078789296131}
{'ner': 7.918983608043928}
{'ner': 2.909089909616881}
{'ner': 7.019620747849132}
{'ner': 8.323473524633414}
{'ner': 0.016159948286468745}
{'ner': 2.0186607484219654}
{'ner': 2.0390488855665203}
{'ner': 0.010336379984821547}
{'ner': 0.8741607160762073}
{'ner': 0.896305416735632}
{'ner': 0.

In [25]:
for text, _ in TRAIN_DATA:
    print(text)
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

In bar chart show me last 7 months of loss for product ppr.
Entities [('bar chart', 'CHART'), ('last 7 months', 'TIME'), ('product ppr', 'PRODUCT')]
Show me last 4 months sale of product pqr as bar chart.
Entities [('last 4 months', 'TIME'), ('product pqr', 'CHART'), ('bar chart', 'CHART')]
show me in violin chart the last 22 months of expenditure for product ppz
Entities [('violin chart', 'CHART'), ('last 22 months', 'TIME'), ('product ppz', 'PRODUCT')]
provide me product xyz orders in last 3 months in bar chart format 
Entities [('product xyz', 'PRODUCT'), ('last 3 months', 'TIME'), ('bar chart', 'CHART')]
For last 56 months get me in pie chart sales of product aab
Entities [('last 56 months', 'TIME'), ('pie chart', 'CHART'), ('product aab', 'PRODUCT')]
get me product lmno sales for last 18 months in violin chart
Entities [('product lmno', 'PRODUCT'), ('last 18 months', 'TIME'), ('violin chart', 'CHART')]
provide for product lmp in pie chart last 6 months of sales
Entities [('product

In [33]:
text='product zzorq sales in ham chart for last 36 months '
#text='give sale of xyz'

In [34]:
doc = nlp(text)

In [35]:
print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

Entities [('product zzorq', 'PRODUCT'), ('ham chart', 'CHART'), ('last 36 months', 'TIME')]


In [40]:
doc

product zzorq sales in ham chart for last 36 months 

In [38]:
doc.text

'product zzorq sales in ham chart for last 36 months '

In [39]:
doc.ents

(product zzorq, ham chart, last 36 months)

In [29]:
out_dir= dataiku.Folder("saves")
out_dir_info = out_dir.get_info()
out_dir_path=saves_info['path']+'/first_test_model'

In [30]:
nlp.to_disk(out_dir_path)

In [67]:
!python --version

Python 3.6.8


In [68]:
!python import spacy

python: can't open file 'import': [Errno 2] No such file or directory


In [39]:
!python -m spacy init fill-config saves_path config.cfg

/usr/bin/python: No module named spacy


In [42]:
import subprocess

In [54]:
command="python -m spacy init fill-config saves_path out"

In [55]:
subprocess.run(command,shell=True,check=True)

CalledProcessError: Command 'python 3+2' returned non-zero exit status 2.

In [61]:
from pathlib import Path
from spacy.cli.download import download
from spacy.cli.init_config import fill_config
from spacy.cli.train import train

In [62]:
fill_config(Path(out_path), Path(saves_path))

ValueError: [E002] Can't find factory for 'transformer' for language English (en). This usually happens when spaCy calls `nlp.create_pipe` with a custom component name that's not registered on the current language class. If you're using a Transformer, make sure to install 'spacy-transformers'. If you're using a custom component, make sure you've added the decorator `@Language.component` (for function components) or `@Language.factory` (for class components).

Available factories: attribute_ruler, tok2vec, merge_noun_chunks, merge_entities, merge_subtokens, token_splitter, doc_cleaner, parser, beam_parser, lemmatizer, trainable_lemmatizer, entity_linker, entity_ruler, tagger, morphologizer, ner, beam_ner, senter, sentencizer, spancat, spancat_singlelabel, span_finder, future_entity_ruler, span_ruler, textcat, textcat_multilabel, en.lemmatizer

In [0]:
train(Path("config.cfg"), Path("my_model"), overrides={"paths.train": "train.spacy", "paths.dev": "dev.spacy"})